In [1]:
import os
import json
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch
from torchvision import models
import torch.nn as nn
from tqdm import tqdm


import os
import json
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch
from torchvision import models
import torch.nn as nn
from tqdm import tqdm

In [2]:
class CustomDataset(Dataset):
    def __init__(self, data_dir="test_imgs", transform=None, expected_size=(224, 224)):
        self.transform = transform
        self.image_paths = []
        # self.labels = []
        # for dir in os.listdir(data_dir):
        for i in os.listdir(os.path.join(data_dir)):
                if '.png' in i or '.jpg' in i:
                    self.image_paths.append(os.path.join(data_dir, i))
                # self.labels.append(i.split('_')[0])
                # print("Label:", self.labels[-1])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        # print(img_path)
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)
        # label = self.labels[idx]
        # label = label.replace('.', ',')
        # label = [float(label)]
        # label = torch.Tensor(label)
        return [image, img_path]

In [3]:
# model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)


In [12]:
import torch
import torchvision.transforms.functional as F
from PIL import Image

class PadToSquare:
    def __call__(self, img):
        """
        Pad the input image tensor to a square shape by adding zeros to the smaller dimension.
        Args:
            img (PIL Image or Tensor): Input image.
        Returns:
            PIL Image or Tensor: Padded image to square shape.
        """
        if isinstance(img, torch.Tensor):
            c, h, w = img.shape
        elif isinstance(img, Image.Image):
            w, h = img.size
        else:
            raise TypeError("Input should be a PIL Image or a torch.Tensor")
        
        max_side = max(h, w)
        
        # Calculate padding sizes
        pad_h = max_side - h
        pad_w = max_side - w
        
        if isinstance(img, torch.Tensor):
            padded_img = F.pad(img, (0, 0, pad_w, pad_h), fill=0)
        elif isinstance(img, Image.Image):
            padded_img = F.pad(img, (0, 0, pad_w, pad_h), fill=0)
        
        return padded_img


In [13]:
transform = transforms.Compose([
    # transforms.Resize((1024, 1024)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    PadToSquare(),
])
device = 'cuda'

In [14]:
import torch.nn as nn
from doctr.models import db_resnet50

model_back = db_resnet50(pretrained=True)
# model = model.feat_extractor
# for param in model.parameters():
#     param.requires_grad = False
del model_back.thresh_head
del model_back.prob_head

class MyModel(nn.Module):
    def __init__(self, model_back):
        super(MyModel, self).__init__()
        self.feat_extractor = model_back

    def forward(self, x):
        features = self.feat_extractor(x)
        print("shape features", features.shape)
        
        # Concatenate FPN outputs
        concat_features = torch.cat(features, dim=1)
        print("shape concated:", concat_features.shape)

        return concat_features

model = MyModel(model_back)


# new_layer = nn.Sequential(
#     nn.Linear(in_features=2048, out_features=1024, bias=True),
#     nn.Linear(in_features=1024, out_features=1, bias=True)
# )
# model.add_module("fc", new_layer)

# print(model)

In [15]:
model

MyModel(
  (feat_extractor): DBNet(
    (feat_extractor): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  

In [18]:
model.to(device)
print(inputs.shape)
answ = model(inputs)
answ

torch.Size([1, 3, 1000, 1000])


RuntimeError: The size of tensor a (64) must match the size of tensor b (63) at non-singleton dimension 3

In [17]:
test_imgs_folder = "test_imgs"

for subfolder in os.listdir(test_imgs_folder):
    dataset = CustomDataset(data_dir=os.path.join(test_imgs_folder, subfolder), transform=transform)
    dataloader = DataLoader(dataset, batch_size=1, shuffle=False, pin_memory=True)

    model.to(device)
    results = dict()
    for img in dataloader:
        inputs = img[0].to(device)
        # inputs = torch.transpose(inputs, 1, 3)
        # inputs = torch.reshape(inputs, (1, 3, 1024, 1024))
        outputs = model(inputs)
        results.update({img[1][0]: outputs[0][0].item()})
        # print(img[1][0])
        # print(outputs[0][0].item())

    results = sorted(results.items(), key=lambda item: item[1], reverse=True)
    for key, value in results:
        print(f"{key}: {value}")

    print("-"*30, '\n')


RuntimeError: The size of tensor a (64) must match the size of tensor b (63) at non-singleton dimension 3

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset, random_split

# Определите устройство (GPU, если доступно, иначе CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Подготовка данных
transform = transforms.Compose([
    # transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Создание датасета и загрузчика данных
dataset = CustomDataset(transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])


In [ ]:
num_epochs = 5
losses = []
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss().to(device)
model.to(device)

dataloaders = {
        'train': DataLoader(train_dataset, batch_size=1, shuffle=True, pin_memory=True),
        'val': DataLoader(test_dataset, batch_size=1, shuffle=False, pin_memory=True)
    }

In [ ]:
for epoch in range(num_epochs):
    print(f'Epoch {epoch}/{num_epochs - 1}')
    print('-' * 10)
    for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                # labels = labels.to(device).unsqueeze(1)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            print(f'{phase} Loss: {epoch_loss:.4f}')

In [ ]:
model.eval()
with torch.no_grad():
    for img, lbl in DataLoader(test_dataset, batch_size=1, shuffle=False, pin_memory=True):
        img = img.to(device)
        lbl = lbl.to(device)
        outputs = model(img)
        loss = criterion(outputs, lbl)
        print(round(outputs[0][0].item(), 2), round(lbl[0][0].item(), 2), loss.item())
        

In [ ]:
torch.save(model.state_dict(), 'model_ft.pt')